## Connect to Snowflake 

In [21]:
import os
import json
import snowflake.connector
import pandas as pd


## Load the Credentials into Environment Variables

In [10]:
with open('db_config.json') as config_file:
    config = json.load(config_file)

# Set environment variables
os.environ['SNOWFLAKE_ACCOUNT'] = config['account']
os.environ['SNOWFLAKE_USER'] = config['user']
os.environ['SNOWFLAKE_PASSWORD'] = config['password']
os.environ['SNOWFLAKE_WAREHOUSE'] = config['warehouse']
os.environ['SNOWFLAKE_DATABASE'] = config['database']
os.environ['SNOWFLAKE_SCHEMA'] = config['schema']

print("Environment variables set successfully.")

Environment variables set successfully.


## Establish a Connection to Snowflake

In [11]:
# Establish a connection to Snowflake using environment variables
conn = snowflake.connector.connect(
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
    database=os.getenv('SNOWFLAKE_DATABASE'),
    schema=os.getenv('SNOWFLAKE_SCHEMA')
)

print("Connection to Snowflake established successfully.")


Connection to Snowflake established successfully.


## Create a Function to Execute & Fect SQL Queries then Close the Conn

In [33]:
# Function to execute a query
def execute_query(query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        print("Query executed successfully.")
    except Exception as e:
        print(f"Error executing query: {e}")
    finally:
        cursor.close()

def fetch_records_as_df(query):
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        # Fetching the column names
        columns = [desc[0] for desc in cursor.description]
        # Fetching all records
        records = cursor.fetchall()
        # Creating a DataFrame
        df = pd.DataFrame(records, columns=columns)
        return df
    except Exception as e:
        print(f"Error fetching records: {e}")
    finally:
        cursor.close()

## Create a Sample Table

In [14]:
create_schema_query = """
CREATE SCHEMA IF NOT EXISTS DEV_ANALYTICS;
"""
execute_query(create_schema_query)


create_table_query = """
CREATE TABLE IF NOT EXISTS DEV_ANALYTICS.STUDENTS (
    ID INT PRIMARY KEY,
    NAME STRING,
    AGE INT,
    CITY STRING,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
execute_query(create_table_query)


Query executed successfully.
Query executed successfully.


## Insert records into the STUDENTS table

In [15]:

insert_records_query = """
INSERT INTO DEV_ANALYTICS.STUDENTS (ID, NAME, AGE, CITY) VALUES
    (1, 'Alex', 20, 'Coimbatore'),
    (2, 'Bob', 22, 'Bangalore'),
    (3, 'Charlie', 23, 'Chennai'),
    (4, 'Daniel', 21, 'Hydrabad');
"""
execute_query(insert_records_query)


Query executed successfully.


In [34]:
select_query = """
SELECT * FROM DEV_ANALYTICS.STUDENTS;
"""

students_df = fetch_records_as_df(select_query)
students_df

,ID,NAME,AGE,CITY,CREATED_AT
0,1,Alex,20,Coimbatore,2024-11-05 00:21:53.010
1,2,Bob,22,Bangalore,2024-11-05 00:21:53.010
2,3,Charlie,23,Chennai,2024-11-05 00:21:53.010
3,4,Daniel,21,Hydrabad,2024-11-05 00:21:53.010
